<a href="https://colab.research.google.com/github/nurulqa/Case-Study-03-Kelompok-H/blob/main/Study_Case_3_Kelompok_H.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1><strong><font color="green">Study Case 03 - Kelompok H</font></strong></h1></center>

<img alt="" src="images/covers/cover_ui-n-taudata.jpg"/>

1. Aisyah Ghefira (2006572945)
2. Andini Assyahidah (2006571040)
3. Dewita Oktavia Nuur Marwan (2006531195)
4. Kezia Emmanuela (2006572964)
5. Maria Angelica Christabel (2006483593)
6. Nafisya Alya Aurelitha (2006571122)
7. Nurul Qomariah Abdillah (2006463811)

# PreProcessing Data

In [ ]:
# Warning install "mlxtend" module first. Jalankan cell ini Hanya jika menggunakan Google Colaboratory:
# Jika menggunakan Jupyter notebook secara lokal (e.g. Anaconda/WinPython) silahkan install lewat terminal (e.g. command prompt)
try:
    import google.colab as gc_
    print("Running the code in Google Colab.", gc_) #IN_COLAB = True
    print("Installing required Module, please wait ... ")
    !pip install mlxtend 
    !pip install pycaret
except:
    print("Running the code locally, make sure to install required modules.")#IN_COLAB = False

Running the code in Google Colab. <module 'google.colab' from '/usr/local/lib/python3.7/dist-packages/google/colab/__init__.py'>
Installing required Module, please wait ... 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import warnings; warnings.simplefilter('ignore')
import pandas as pd, matplotlib.pyplot as plt, seaborn as sns
from itertools import combinations
from collections import Counter
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from pycaret.arules import *

%matplotlib inline
plt.style.use('bmh'); sns.set()

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
#IMPORT DATA
df =  pd.read_csv ("https://raw.githubusercontent.com/nurulqa/Case-Study-03-Kelompok-H/main/cae_Kasus-AR-MBA-Ganjil-2022.csv")
df

In [ ]:
#FILL NA
df['Konsumen'].fillna(method = 'ffill', inplace = True)
df['Tanggal'].fillna(method = 'ffill', inplace=True)
df['ID Transaksi'].fillna(method = 'ffill', inplace=True)
df['Tanggal'] = pd.to_datetime(df['Tanggal'])
df

In [ ]:
# cek missing values
df.info()

# **DATA WRANGLING dan EDA**

In [ ]:
# Filter top scoring
df_top = df[df['Jumlah'] >= max(df['Jumlah'])]
df_top

Dapat dilihat, dalam sekali transaksi, produk yang terbanyak dibeli yaitu Snack Citata dan Mie Instan Indomi dengan masing- masing 3 item terjual dibeli oleh Mr.B.

In [ ]:
df['Jumlah'].groupby([df['Nama Produk']]).sum()

In [ ]:
#Frekuensi pembelian produk
import seaborn as sns
import matplotlib.pyplot as plt
p = sns.countplot(x="Nama Produk", data=df)

Dapat dilihat total produk yang paling banyak terjual yaitu Air Minum dengan total 16 item, sedangkan yang paling sedikit terjual yaitu Rokok dengan 5 item terjual.

In [ ]:
df.groupby(['Nama Produk', 'Merek'])[['Jumlah']].sum()

Dari hasil diatas, dapat diketahui produk yang paling banyak terjual yaitu Roti Sri Roti dan Snack Chiki dengan masing- masing 11 item terjual. Produk yang paling sedikit terjual yaitu Roti Prambanan dan Sabun Luks dengan masing- masing 4 item terjual.

In [ ]:
pd.crosstab(df.Konsumen, df.Merek)

In [ ]:
df.groupby(['Konsumen', 'ID Produk'])[['Jumlah']].sum()

In [ ]:
plt.figure(figsize=(8,6))
p = sns.countplot(x="Konsumen",data=df)
#plt.plot(x, y)

Dari hasil diatas, dapat dilihat bahwa pembelian terbanyak dilakukan oleh Mr.B dengan total pembelian 8 produk dan pembelian sedikit dilakukan oleh Mr.D, Mr.P, dan Mr.J dengan masing- masing pembelian 1 produk.

# **MARKET BASKET ANALYSIS**

In [ ]:
#Mengganti tipe data
import datetime
df['Tanggal'] = pd.to_datetime(df['Tanggal'])
df['Konsumen'] = df['Konsumen'].astype('category')
df['Konsumen'] = df['Konsumen'].astype('category')
df['Nama Produk'] = df['Nama Produk'].astype('category')
df['Merek'] = df['Merek'].astype('category')

In [ ]:
#Cek Missing Value
df.isnull().sum().sort_values(ascending=False) 

VISUALISASI DATA

In [ ]:
import numpy as np

#TOP 10 PRODUK TERJUAL

#Membuat Distribusi dari Item Terjual
Item_distr = df.groupby(by = "Nama Produk").size().reset_index(name='Jumlah').sort_values(by = 'Jumlah',ascending=False).head(10)

#Mendeklarasikan Variabel
bars = Item_distr["Nama Produk"]
height = Item_distr["Jumlah"]
x_pos = np.arange(len(bars))

#Ukuran Grafik

plt.figure(figsize=(16,9))

# Membuat Visualisasi
plt.bar(x_pos, height, color=(0.3, 0.4, 0.6, 0.6))

# Memberi judul dan label grafik
plt.title("Top 10 Sold Items")
plt.xlabel("Nama Produk")
plt.ylabel("Jumlah Produk Terjual")

# Membuat nama pada sumbu x
plt.xticks(x_pos, bars)

# Memunculkan grafik
plt.show()

Dari grafik di atas, dapat dilihat bahwa produk paling banyak terjual adalah air minum dan produk paling sedikit terjual adalah rokok.

In [ ]:
#VISUALISASI JUMLAH PRODUK TERJUAL SETIAP BULAN
#Set data sebagai index untuk visualisasi data

df_date=df.set_index(['Tanggal']) ## Setting date as index for plotting purpose
df_date

In [ ]:
#VISUALISASI JUMLAH PRODUK TERJUAL SETIAP BULAN

df_date.resample("M")['Nama Produk'].count().plot(figsize = (20,8), grid = True, title = "Jumlah Produk Terjual Setiap Bulan").set(xlabel = "Tanggal", ylabel = "Jumlah Produk Terjual")

Dapat dilihat bahwa jumlah produk yang terjual menurun drastis dari Juli 2022 hingga Agustus 2020, lalu meningkat sedikit pada September 2022, dan menurun kembali pada Oktober 2022.

In [ ]:
#GRAFIK PEMBELIAN BERDASARKAN WAKTU

plt.figure(figsize=(10,5))
df.groupby('Tanggal')['ID Transaksi'].count().plot()

Untuk lebih detailnya, dapat dilihat pembelian menurun drastis hingga selang waktu 1 Juli 2022 - 15 Juli 2022, lalu konstan hingga 1 Agustus 2022, lalu naik hingga selang waktu 1 September 2022 - 15 September 2022, dan menurun kembali hingga 1 Oktober 2022.

In [ ]:
#TOP 10 MEREK TERJUAL

# Membuat distribusi dari merek terjual
Item_distr1 = df.groupby(by = "Merek").size().reset_index(name='Jumlah').sort_values(by = 'Jumlah',ascending=False).head(10)

# Mendeklarasikan variabel
bars = Item_distr1["Merek"]
height = Item_distr1["Jumlah"]
x_pos = np.arange(len(bars))

# Ukuran Grafik
plt.figure(figsize=(16,9))

# Membuat Bar
plt.bar(x_pos, height, color=(0.3, 0.4, 0.6, 0.6))

# Menambahkan label
plt.title("Top 10 Merek Terjual")
plt.xlabel("Merek Produk")
plt.ylabel("Jumlah Merek Terjual")

# Membuat nama pada sumbu x
plt.xticks(x_pos, bars)

# Memunculkan grafik
plt.show()

Dapat dilihat bahwa merek paling banyak terjual adalah Chiki dan produk paling sedikit terjual adalah Mie Sedap.

In [ ]:
#TOP 10 KONSUMEN DENGAN PEMBELIAN TERENDAH

#Membuat distribusi konsumen terhadap jumlah pembelian produk
Item_distr2 = df.groupby(by = "Konsumen").size().reset_index(name='Jumlah').sort_values(by = 'Jumlah',ascending=True).head(10)

#Mendeklarasikan Variabel
bars = Item_distr2["Konsumen"]
height = Item_distr2["Jumlah"]
x_pos = np.arange(len(bars))

#Ukuran Grafik
plt.figure(figsize=(16,9))

#Membuat Grafik
plt.bar(x_pos, height, color=(0.3, 0.4, 0.6, 0.6))

#Menambahkan label Grafik
plt.title("Top 10 Konsumen dengan Pembelian terendah")
plt.xlabel("Konsumen")
plt.ylabel("Jumlah Pembelian")

#Membuat nama pada sumbu x
plt.xticks(x_pos, bars)

#Memunculkan Grafik
plt.show()

Dari grafik terlihat konsumen dengan jumlah pembelian paling tinggi dan rendah di Alfaindo cabang UI Depok bahwa Mr. A yang memiliki frekuensi belanja paling banyak dan Mr.F dan Mr. J yang memiliki frekuensi belanja paling sedikit.

In [ ]:
basket = (df.groupby(['ID Transaksi', 'Nama Produk'])['Jumlah'].count()\
                                      .unstack().reset_index().fillna(0)\
                                      .set_index('ID Transaksi'))
basket.head()

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.head(5)

In [ ]:
basket_sets.columns

In [ ]:
basket_sets.index

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
frequent_itemsets1 = apriori(basket_sets, min_support=0.07, use_colnames=True)
frequent_itemsets1.sort_values(by='support', ascending=False, na_position='last', inplace = True)
frequent_itemsets1

In [ ]:
rules = association_rules(frequent_itemsets1, metric="lift", min_threshold=1)
rules.sort_values(by='lift', ascending=False, na_position='last', inplace = True)
rules.head()

In [ ]:
rules1 = association_rules(frequent_itemsets1, metric="lift", min_threshold=1)
rules1.head()

In [ ]:
# Filtering
rules[ (rules['lift'] >= 6) & (rules['confidence'] >= 0.8)]

Dapat dilihat bahwa kecendrungan yang paling sering terjadi adalah:
1. Bila seseorang membeli Mie Instan, Roti, dan Snack maka mereka akan cenderung membeli Sabun dan Air Minum
2. Bila seseorang membeli Air Minum, Mie Instan, dan Snack maka mereka akan cenderung membeli Sabun dan Roti

In [ ]:
basket1 = (df.groupby(['ID Transaksi', 'Merek'])['Jumlah'].count()\
                                      .unstack().reset_index().fillna(0)\
                                      .set_index('ID Transaksi'))
basket1.head()

In [ ]:
frequent_itemsets1 = apriori(basket1, min_support=0.1, use_colnames=True)
frequent_itemsets1

In [ ]:
rules2 = association_rules(frequent_itemsets1, metric="lift", min_threshold=1)

In [ ]:
result2 = rules2[(rules1['lift'] >= 1) & 
               (rules1['confidence'] >= 0.8)]

apr_result1 = result2.sort_values(by='confidence', ascending=False)
apr_result1.head(20)

1. Kolom Antecedent Support menyatakan probabilitas konsumen hanya membeli produk antecedents saja. Misal, probabilitas konsumen hanya membeli Roti adalah 50%.
2. Kolom Consequent Support menyatakan probabilitas konsumen hanya membeli produk consequents saja. Misal, probabilitas konsumen hanya membeli Air Minum adalah 61,11%.
3. Kolom Support menyatakan probabilitas konsumen membeli produk antecedents dan consequents. Misal, probabilitas konsumen membeli Roti dan Air Minum adalah 44,44%
4. Kolom Confidence menyatakan ukuran seberapa sering aturan (1), (2), dan (3) terbukti benar.
5. Kolom Lift menyatakan ukuran seberapa sering produk antecedents dan consequents dibeli secara bersamaan dibandingkan jika dibeli secara tidak bersmaaan.

In [ ]:
apr_result1['consequents'].value_counts()

Data di atas merupakan data merek produk dimana konsumen akan membeli merek produk tersebut apabila ia membeli merek produk tertentu. Dapat dilihat bahwa merek produk yang paling banyak dibeli adalah Aquos dan merek produk yang paling sedikit dibeli adalah Sri Roti, Sri Roti dan Sprit, Luks, Clean, Indomi dan Sri Roti, Sri Roti dan Lifboy, Malioboro, dan Mi Sedap.

# **Rekomendasi dan Kesimpulan**
Rekomendasi dan kesimpulan:
-	Disarankan kepada data engineering untuk lain kali tidak menyimpan data di excel melainkan menyimpannya di csv. Hal ini dikarenakan mempermudah pengerjaan data analyst dalam memproses data serta untuk mengantisipasi scale up business. Hal ini penting dikarenakan excel memiliki kelemahan fatal yaitu memiliki jumlah baris yang terbatas sehingga jika tetap menggunakan excel saat member loyalty programnya semakin banyak maka terdapat kemungkinan beberapa data pelanggan tersebut dapat terhapus.
-	Direkomendasikan kepada perusahaan untuk menambah stok dan variasi produk yang terjual lebih banyak yaitu Air minum dan roti agar penjualan terus meningkat dikemudian hari. 
-	Direkomendasikan kepada perusahaan untuk menjual rokok dengan merek yang lain agar lebih menarik pembeli dikarenakan rokok merupakan produk yang paling sedikit terjual.
-	Direkomendasikan kepada perusahaan untuk memberikan promo pada produk rokok agar penjualan produk rokok meningkat dikarenakan penjualan produk rokok lebih sedikit dibandingkan dengan produk lain.
-	Direkomendasikan kepada perusahaan untuk memberikan reward kepada pelanggan yang paling banyak membeli produk agar pelanggan tersebut dan pelanggan lainnya termotivasi untuk membeli produk lebih banyak sehingga bisa mendapatkan reward yang diberikan oleh perusahaan.
-	Direkomendasikan kepada perusahaan untuk memvariasikan produk yang dijual agar pembeli memiliki banyak pilihan dalam membeli produk di perusahaan tersebut.
